In [11]:
import pandas as pd

employees_old = r"C:\Users\Narasimha\Desktop\ai_recruiting_infrastructure_agent2\scrapers\data\raw\employees_past_week.csv"
employees_new=r"C:\Users\Narasimha\Desktop\ai_recruiting_infrastructure_agent2\scrapers\data\raw\employees_present_week.csv"

old_df = pd.read_csv(employees_old, index_col=False)
# Merge into last column
old_df.iloc[:, 4] = old_df.iloc[:, 4:].astype(str).apply(lambda x: ",".join(x.dropna()), axis=1)

# Drop all columns after the 5th
#old_df = old_df.iloc[:, :5]

# Rename
old_df.columns = ["email_id", "name", "role", "company_name", "company_location"]

new_df = pd.read_csv(employees_new, index_col=False)
new_df.iloc[:, 4] = new_df.iloc[:, 4:].astype(str).apply(lambda x: ",".join(x.dropna()), axis=1)

# Drop all columns after the 5th
#new_df = new_df.iloc[:, :5]

# Rename
new_df.columns = ["email_id", "name", "role", "company_name", "company_location"]

"""Make joins/comparisons robust (strip + handle NaN)
CSV data often has:
trailing spaces
NaN values
inconsistent casing """
for df in (old_df, new_df):
    df["email_id"] = df["email_id"].astype(str).str.strip().str.lower()
    df["role"] = df["role"].astype(str).str.strip()
    df["company_name"] = df["company_name"].astype(str).str.strip()



#main logic starts from here

merged = old_df[["email_id", "role", "company_name", "company_location"]].merge(
    new_df[["email_id", "role", "company_name"]],
    on="email_id",
    how="inner",
    suffixes=("_old", "_new")
).fillna("")

changed_rows = merged[
    (merged["company_name_old"] != merged["company_name_new"]) | 
    (merged["role_old"] != merged["role_new"])
    ].copy()

changed_rows["status"] = changed_rows.apply(
    lambda row: "company changed"
    if row["company_name_old"] != row["company_name_new"]
    else "role changed",
    axis=1
)

changed_rows["company"] = (
    changed_rows["company_name_old"]
    + " (" + changed_rows["company_location"] + ")"
)

new_dataset = changed_rows[
    ["email_id", "company", "role_old", "status"]
]

new_dataset = new_dataset.reset_index(drop=True)
new_dataset.index = new_dataset.index + 1

new_dataset.columns=["Email", "Company (Location)", "Position", "Status"]


print(new_dataset)

                     Email             Company (Location)  \
1         chris.p@apex.com        Apex Solutions (Dublin)   
2     sophia.t@everest.com    Everest Consulting (Zurich)   
3    victoria.n@lumina.com      Lumina Designs (New York)   
4        mark.d@fusion.com        Fusion Robotics (Tokyo)   
5      paula.s@quantum.com         Quantum AI (Singapore)   
6       brian.c@altair.com      Altair Security (Chicago)   
7      rachel.p@cypher.com             Cypher AI (London)   
8     hailey.d@monarch.com    Monarch Industries (London)   
9        colin.b@astro.com          Astro Cloud (Chicago)   
10      ariana.l@polar.com        Polar Studios (Toronto)   
11   sienna.b@crescent.com  Crescent Solutions (New York)   
12    logan.k@serenity.com       Serenity Health (London)   
13       laura.d@avant.com              Avant BI (London)   
14    julian.h@maxwell.com  Maxwell Investments (Toronto)   
15  tristan.h@datacore.com              DataCore (Boston)   

                       

C:\Users\Narasimha\AppData\Local\Temp\ipykernel_21292\3439141507.py:6: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  old_df = pd.read_csv(employees_old, index_col=False)
C:\Users\Narasimha\AppData\Local\Temp\ipykernel_21292\3439141507.py:16: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  new_df = pd.read_csv(employees_new, index_col=False)


In [13]:
import pandas as pd

employees_old = r"C:\Users\Narasimha\Desktop\ai_recruiting_infrastructure_agent2\scrapers\data\raw\employees_past_week.csv"
employees_new = r"C:\Users\Narasimha\Desktop\ai_recruiting_infrastructure_agent2\scrapers\data\raw\employees_present_week.csv"

# -------------------------
# CONFIG
# -------------------------
USECOLS = [0, 1, 2, 3, 4]  # only first 5 columns
DTYPES = {
    0: "string",
    1: "string",
    2: "string",
    3: "string",
    4: "string"
}

# -------------------------
# LOAD CSVs (FAST)
# -------------------------
old_df = pd.read_csv(
    employees_old,
    usecols=USECOLS,
    dtype=DTYPES,
    engine="c",
)

new_df = pd.read_csv(
    employees_new,
    usecols=USECOLS,
    dtype=DTYPES,
    engine="c",
)

old_df.columns = ["email_id", "name", "role", "company_name", "company_location"]
new_df.columns = ["email_id", "name", "role", "company_name", "company_location"]

# -------------------------
# CLEAN DATA (VECTORISED)
# -------------------------
for df in (old_df, new_df):
    df["email_id"] = df["email_id"].str.strip().str.lower()
    df["role"] = df["role"].str.strip()
    df["company_name"] = df["company_name"].str.strip()
    df["company_location"] = df["company_location"].str.strip().fillna("")

# -------------------------
# MERGE ONLY NEEDED COLUMNS
# -------------------------
merged = old_df[
    ["email_id", "role", "company_name", "company_location"]
].merge(
    new_df[["email_id", "role", "company_name"]],
    on="email_id",
    how="inner",
    suffixes=("_old", "_new")
)

# -------------------------
# FILTER CHANGED ROWS (VECTORIZED)
# -------------------------
company_changed = merged["company_name_old"] != merged["company_name_new"]
role_changed = merged["role_old"] != merged["role_new"]

changed_rows = merged[company_changed | role_changed]

# -------------------------
# STATUS (NO APPLY)
# -------------------------
changed_rows["Status"] = "role changed"
changed_rows.loc[company_changed, "Status"] = "company changed"

# -------------------------
# FINAL REPORT
# -------------------------
changed_rows["Company (Location)"] = (
    changed_rows["company_name_old"]
    + " ("
    + changed_rows["company_location"].replace("", "Unknown")
    + ")"
)

new_dataset = changed_rows[
    ["email_id", "Company (Location)", "role_old", "Status"]
].rename(
    columns={
        "email_id": "Email",
        "role_old": "Position"
    }
)

new_dataset = new_dataset.reset_index(drop=True)
new_dataset.index += 1

print(new_dataset)


                     Email             Company (Location)  \
1         chris.p@apex.com        Apex Solutions (Dublin)   
2     sophia.t@everest.com    Everest Consulting (Zurich)   
3    victoria.n@lumina.com      Lumina Designs (New York)   
4        mark.d@fusion.com        Fusion Robotics (Tokyo)   
5      paula.s@quantum.com         Quantum AI (Singapore)   
6       brian.c@altair.com      Altair Security (Chicago)   
7      rachel.p@cypher.com             Cypher AI (London)   
8     hailey.d@monarch.com    Monarch Industries (London)   
9        colin.b@astro.com          Astro Cloud (Chicago)   
10      ariana.l@polar.com        Polar Studios (Toronto)   
11   sienna.b@crescent.com  Crescent Solutions (New York)   
12    logan.k@serenity.com       Serenity Health (London)   
13       laura.d@avant.com              Avant BI (London)   
14    julian.h@maxwell.com  Maxwell Investments (Toronto)   
15  tristan.h@datacore.com              DataCore (Boston)   

                       

C:\Users\Narasimha\AppData\Local\Temp\ipykernel_21292\1387250574.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed_rows["Status"] = "role changed"
C:\Users\Narasimha\AppData\Local\Temp\ipykernel_21292\1387250574.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  changed_rows["Company (Location)"] = (


In [4]:
import pandas as pd
k=r"employees_past_week.csv"
USECOLS = [0, 1, 2, 3, 4]  # only first 5 columns
DTYPES = {
    0: "string",
    1: "string",
    2: "string",
    3: "string",
    4: "string"
}
#df= pd.read_
df = pd.read_csv(
    k,
    index_col=False,
    usecols=USECOLS,
    dtype=DTYPES,
    engine="c",
)

df.columns = ["email_id", "name", "role", "company_name", "company_location"]

In [5]:
df.head()

,email_id,name,role,company_name,company_location
0,john.doe@acme.com,John Doe,Software Engineer,ACME Corporation,San Francisco
1,jane.smith@globex.com,Jane Smith,Product Manager,Globex Inc.,New York
2,robert.j@initech.com,Robert Johnson,Data Analyst,Initech,Boston
3,emily.c@umbrella.com,Emily Clark,Marketing Director,Umbrella Corp,London
4,michael.b@hooli.com,Michael Brown,CTO,Hooli,Palo Alto
